In [3]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
%matplotlib inline

In [110]:
def matrix_vector_product(A, x):
    """
    Calculates matrix-vector product.
    Args:
        A (array_like): A m-by-n matrix
        x (array_like): Vector of size n
    Returns:
        ndarray: Matrix-vector product
    """
    m, n = A.shape
    b = np.zeros(m)
    for i in range(m):
        for j in range(n):
            b[i] += A[i, j] * x[j]
    return b

In [111]:
def matrix_matrix_product(A, B):
    """
    Calculates matrix-matrix product.
    Args:
        A (array_like): A m-by-n matrix
        B (array_like): A n-by-p matrix
    Returns:
        ndarray: Matrix-matrix product
    """
    m, n = A.shape
    n, p = B.shape
    C = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]
    return C

In [105]:
def forward_substitution(A, b):
    """
    Solves a system of linear equation with lower triangular matrix.
    Args:
        A (array_like): A n-by-n lower triangular matrix
        b (array_like): RHS vector of size n
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    for i in range(0, n):
        x[i] = (b[i] - np.dot(A[i, :], x)) / A[i, i]
    return x

In [106]:
def backward_substitution(A, b):
    """
    Solves a system of linear equation with upper triangular matrix.
    Args:
        A (array_like): A n-by-n upper triangular matrix
        b (array_like): RHS vector of size n
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, :], x)) / A[i, i]
    return x

In [8]:
def gaussian_elimination(A, b):
    """
    Transform given matrix into upper triangular form, perform identical operations on RHS vector.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
    Returns:
        ndarray: Upper triangular matrix
        ndarray: RHS vector corresponding to upper triangular matrix
    """
    n, n = A.shape
    tmp = np.zeros((n, n+1))
    tmp[:,:-1] = A
    tmp[:,-1] = b
    for i in range(0, n):  
        # return the index of max. row relatively to i (max_row 0 means that the i-th row has maximum)
        max_row = np.argmax(A[i:, i])
        if (max_row != 0):
            row_i = np.copy(tmp[i, :])
            tmp[i, :] = np.copy(tmp[i + max_row, :])
            tmp[i + max_row, :] = np.copy(row_i)    
        for j in range(i + 1, n):
            tmp[j, :] = tmp[j, :] - (tmp[j, i] / tmp[i, i]) * tmp[i, :]      
    return tmp[:,:-1], tmp[:, -1]

In [47]:
def lu_decomposition(A):
    """
    Decompose given matrix into a product of a lower and an upper triangular matrix.
    Args:
        A (array_like): A n-by-n regular matrix
    Returns:
        ndarray: Lower triangular matrix
        ndarray: Upper triangular matrix
    """
    n, n = A.shape
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    for i in range(0, n):
        for j in range(i, n):
            U[i, j] = A[i, j] - np.dot(L[i, :], U[:, j])
        for j in range(i, n):
            L[j, i] = (A[j, i] - np.dot(L[j, :], U[:, i])) / U[i, i]
    return L, U

In [10]:
def thomas_algorithm(A, f):
    """
    Solves system of linear equations with tridiagonal matrix using Thomas' algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        f (array_like): RHS vector of size n
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape # get the size of input matrix
    c = np.diag(A, -1) # get elements below diagonal 
    a = np.diag(A, 0) # get elements on diagonal
    b = np.diag(A, 1) # get elements above diagonal
    c = np.insert(c, 0, 0.) # insert 0 as a first element of c
    b = np.insert(b, b.size, 0.) # insert 0 as a last element of b
    x = np.zeros(n+1) 
    rho = np.zeros(n+1); 
    mu  = np.zeros(n+1);
    for i in range(0, n):
        mu[i] = -b[i] / (c[i] * mu[i-1] + a[i])
        rho[i] = (f[i] - c[i] * rho[i-1]) / (c[i] * mu[i-1] + a[i])
    for i in range(n-1, -1, -1):
        x[i] = mu[i] * x[i+1] + rho[i];
    return x[:-1]

In [100]:
def jacobi_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using Jacobi's algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)   
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i, i]) * (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,i+1:], x[i+1:]))       
        if(la.norm(np.dot(A, x_new) - b) < eps):
            x = x_new
            break       
        x = x_new
    return x

In [101]:
def gauss_seidel_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using Gauss-Seidel's algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)  
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x_new[:i]) - np.dot(A[i,i+1:], x[i+1:]))      
        if(la.norm(np.dot(A, x_new) - b) < eps):
            x = x_new
            break  
        x = x_new
    return x

In [103]:
def successive_overrelaxation_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using successive overrelaxation (SOR) method.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)
    L = np.tril(A, -1) # get lower triangular matrix with zeros on diagonal
    U = np.triu(A, 1) # get upper triangular matrix with zeros on diagonal
    D = A - L - U # get diagonal matrix
    B = -np.dot(la.inv(D + L), U) # calculate iteration matrix
    rho = np.max(np.abs(la.eigvals(B))) # find spectral radius (i.e. maximal eigenvalue in absolute value)
    omega = 2.0 / (1.0 + np.sqrt(np.abs(1.0 - rho**2))) # find optimal relaxation factor
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x_new[:i]) - np.dot(A[i,i+1:], x[i+1:]))
        if(la.norm(np.dot(A, x_new) - b) < eps):
            x = x_new
            break
        x += omega * (x_new - x);
    return x

In [97]:
def power_iteration(A, max_it=500):
    """
    Finds the greatest eigen value (in absolute value) of given matrix and its corresponding eigenvector.
    Args:
        A (array_like): A n-by-n diagonalizable matrix
        max_it (int): Maximum number of iterations
    Returns:
        ndarray: Eigenvector corresponding to a greatest eigenvalue (in absolute value)
        float: Greatest eigenvalue (in absolute value)
    """
    n, n = A.shape
    eigen_vec = np.ones(n)
    for i in range(max_it):
        eigen_vec_new = np.dot(A, v)
        eigen_vec = eigen_vec_new / la.norm(eigen_vec_new)
    eigen_val = la.norm(np.dot(A, v))
    return eigen_vec, eigen_val